In [1]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.5 MB/s eta 0:00:00


In [2]:
import numpy as np 
import tensorflow 
from tensorflow import keras 
import matplotlib as plt
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras import Sequential 
from keras.preprocessing.image import ImageDataGenerator
import keras_tuner
import os
from keras import Sequential
from keras.layers import Dense,Dropout

from os import listdir 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/intel_image/seg_pred/seg_train"
test_data_dir = "/content/drive/MyDrive/Colab Notebooks/intel_image/seg_pred/seg_test/seg_test"

In [16]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_data = image_generator.flow_from_directory(batch_size=64,
                                                 directory=train_data_dir,
                                                 shuffle=True,
                                                 target_size=(64,64), 
                                                 subset="training",
                                                 class_mode='categorical')

Found 11230 images belonging to 6 classes.


In [18]:
val_data = image_generator.flow_from_directory(batch_size=64,
                                                 directory=train_data_dir,
                                                 shuffle=False,
                                                 target_size=(64,64), 
                                                 subset="validation",
                                                 class_mode='categorical')
test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory(test_data_dir,
                                       target_size=(64,64), batch_size=64, shuffle=False)

print(train_data.classes)
print(val_data.classes)


Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
[0 0 0 ... 5 5 5]
[0 0 0 ... 5 5 5]


In [55]:
def build_model1(hp):  
  model1 = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(64,64,3),
        padding = 'same'
    ),
     
    keras.layers.MaxPooling2D((3, 3)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(6, activation='softmax')
  ])

  model1.compile(optimizer='Adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  
  return model1

tuner1 = keras_tuner.RandomSearch(
    build_model1,
    objective='val_accuracy',
    max_trials=10)

tuner1.search(train_data, epochs=5, validation_data=val_data)
tuner1.results_summary()

Trial 10 Complete [00h 02m 23s]
val_accuracy: 0.7571326494216919

Best val_accuracy So Far: 0.7803138494491577
Total elapsed time: 00h 17m 03s
Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
conv_2_filter: 128
conv_2_kernel: 5
learning_rate: 0.001
units: 48
Score: 0.7803138494491577
Trial summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 5
learning_rate: 0.001
units: 120
Score: 0.7710413932800293
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 5
conv_2_filter: 80
conv_2_kernel: 3
learning_rate: 0.005
units: 88
Score: 0.7689015865325928
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 3
conv_2_filter: 112
conv_2_kernel: 3
learning_rate: 0.001
units: 40
Score: 0.7599857449531555
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 5
conv_2_filter: 80
conv_2_kernel: 3
learning_rate: 0.05
units: 104
Score: 0.7571326494

In [61]:
def build_model2(hp):  
    model2 = keras.Sequential([
      keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape = (64, 64, 3), padding = 'same'),
      keras.layers.MaxPooling2D((3, 3)),
      keras.layers.Conv2D(112, (5, 5), activation='relu', padding = 'same'),
      keras.layers.MaxPooling2D((3, 3)),
      keras.layers.Conv2D(80, (5, 5), activation='relu', padding = 'same'),
      keras.layers.MaxPooling2D((3, 3)),
      keras.layers.Flatten(),
      keras.layers.Dense(64, activation = 'relu'),
      keras.layers.Dense(32, activation = 'relu'),
      keras.layers.Dense(6, activation='softmax')
      ])

    model2.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[0.001,0.005,0.025,0.05,0.0005,0.0001])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
  
    return model2

tuner2 = keras_tuner.RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=5)

tuner2.search(train_data, epochs=10, validation_data=val_data)
tuner2.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 5
conv_2_filter: 128
conv_2_kernel: 5
learning_rate: 0.001
units: 48
Score: 0.7803138494491577
Trial summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 5
learning_rate: 0.001
units: 120
Score: 0.7710413932800293
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 5
conv_2_filter: 80
conv_2_kernel: 3
learning_rate: 0.005
units: 88
Score: 0.7689015865325928
Trial summary
Hyperparameters:
conv_1_filter: 112
conv_1_kernel: 3
conv_2_filter: 112
conv_2_kernel: 3
learning_rate: 0.001
units: 40
Score: 0.7599857449531555
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 5
conv_2_filter: 80
conv_2_kernel: 3
learning_rate: 0.05
units: 104
Score: 0.7571326494216919
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 3
learning_rate: 0.0005
units: 104
Sc

In [62]:
model = keras.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape = (64, 64, 3), padding = 'same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding = 'same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'relu'))
model.add(layers.Dense(6, activation = 'softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_data, epochs=10, batch_size=64, validation_data = val_data)
test_loss, test_acc = model.evaluate((test_data))

Epoch 1/10
176/176 [==============================] - 30s 169ms/step - loss: 1.2219 - accuracy: 0.4996 - val_loss: 0.9917 - val_accuracy: 0.6116
Epoch 2/10
176/176 [==============================] - 29s 166ms/step - loss: 0.9299 - accuracy: 0.6339 - val_loss: 0.8668 - val_accuracy: 0.6655
Epoch 3/10
176/176 [==============================] - 29s 164ms/step - loss: 0.8071 - accuracy: 0.6880 - val_loss: 0.7695 - val_accuracy: 0.7079
Epoch 4/10
176/176 [==============================] - 29s 164ms/step - loss: 0.6995 - accuracy: 0.7410 - val_loss: 0.7283 - val_accuracy: 0.7254
Epoch 5/10
176/176 [==============================] - 29s 162ms/step - loss: 0.6185 - accuracy: 0.7778 - val_loss: 0.5920 - val_accuracy: 0.7914
Epoch 6/10
176/176 [==============================] - 31s 175ms/step - loss: 0.5772 - accuracy: 0.7923 - val_loss: 0.5675 - val_accuracy: 0.7921
Epoch 7/10
176/176 [==============================] - 29s 163ms/step - loss: 0.5328 - accuracy: 0.8070 - val_loss: 0.5855 - val_ac

In [63]:
print(test_loss,test_acc)

0.5139740109443665 0.8146666884422302
